# 📢 File explanation!!
In this file, preprocessing of the dataset will be carried out based on the conclusion of information from the file 📈 01_Exploratory Data Analysis

The following is the conclusion of the EDA process

- **Deleted** <br>
  Based on the EDA process, I will remove parts of words that do not represent or support the sentiment category and here are the things I removed
  - link
  - numbers (including dates)
  - `@`
  - `#`
  - emoticons
- **Word replacement** <br>
  In this section, I will replace abbreviations and slang words with real words to make sentences easier to understand.
- There are words that have double letters so they need to be deleted

# 🎯 **Step 0: Import library**
---

In [64]:
!pip install Sastrawi

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import string
import nltk

from itertools import islice
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [65]:
plt.style.use('seaborn-v0_8')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 🎯 **Step 1: Load dataset**
---

## ✨ 1.1 - Dataset tweet

In [67]:
path =  '/content/drive/MyDrive/INDONESIA AI/Sentiment Analysis/Dataset/df_tweet.csv'

df_sentiment = pd.read_csv(path)

df_sentiment.head()

,sentimen,tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...


In [68]:
df_sentiment['tweet'].loc[0]

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

In [69]:
df_sentiment['tweet'].loc[3]

'Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, Jokowi hebat!'

## ✨ 1.2 - Dataset slang words

In [70]:
def load_dataset_slang(path):

  with open(path, 'r') as file:

    slang_words = json.load(file)

  return slang_words


def print_data(data):

  for key, value in islice(data.items(), 5):

    print(key, value)

In [71]:
path = [
    '/content/drive/MyDrive/INDONESIA AI/Sentiment Analysis/Dataset/Slang Words/json_slang_words1',
    '/content/drive/MyDrive/INDONESIA AI/Sentiment Analysis/Dataset/Slang Words/json_slang_words2',
    '/content/drive/MyDrive/INDONESIA AI/Sentiment Analysis/Dataset/Slang Words/json_slang_words3'
]

slang_words1 = load_dataset_slang(path[0])

slang_words2 = load_dataset_slang(path[1])

slang_words3 = load_dataset_slang(path[2])

In [72]:
print_data(slang_words1)

@ di
abis habis
ad ada
adlh adalah
afaik as far as i know


In [73]:
print_data(slang_words2)

aamiin amin 
adek adik 
adlh adalah 
aer air 
aiskrim es krim 


In [74]:
print_data(slang_words3)

woww wow
aminn amin
met selamat
netaas menetas
keberpa keberapa


# 🎯 Step 2: Preprocessing

---


In [75]:
def case_folding_text(text):

    text = text.lower()

    return text

def cleaning_text(text):

    text = re.sub(r'@', '', text)

    text = re.sub(r'#[A-Za-z0-9]+', '', text)

    text = re.sub(r'RT[\s]', '', text)

    text = re.sub(r"http\S+", '', text)

    # text = re.sub(r'[0-9]+', '', text)

    text = re.sub(r'pic\.twitter\.com/[^\s]+', '', text)

    text = re.sub(r'\.+', ' ', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) #remove emoji

    text = text.replace('\n', ' ')

    text = text.replace("'", "")

    text = text.replace(']','')

    text = text.replace('[','')

    text = text.replace(',',', ')

    text = text.replace('/',' ')

    text = text.replace('-',' ')

    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation from a text

    text = text.strip() # remove spaces (whitespace) at the beginning and at the end

    return text

def filtering_slang(text):

    words = text.split()

    for i, word in enumerate(words):

        if word in slang_words1.keys():

            words[i] = slang_words1[word]

        elif word in slang_words2.keys():

            words[i] = slang_words2[word]

        elif word in slang_words3.keys():

            words[i] = slang_words3[word]

    filtered_text = ' '.join(words)

    return filtered_text

def tokenizing_text(text):

    text = word_tokenize(text)

    return text


def remove_duplicate_letters(text):

    cleaned_data = []

    for word in text:

        if word.lower() not in ["assalamualaikum", "bangga"]:

            word = re.sub(r'(.)\1+', r'\1', word)

        cleaned_data.append(word)

    return cleaned_data

def filtering_stopwords(text):

    # listStopwords = set(stopwords.words('indonesian'))

    stop_factory = StopWordRemoverFactory()

    stopword = stop_factory.get_stop_words()

    filtered = []

    for txt in text:

        if txt not in set(stopword) or txt.lower() == 'tidak':

            filtered.append(txt)

    text = filtered

    return text

def stemming_text(text):

    factory = StemmerFactory()

    stemmer = factory.create_stemmer()

    stemmed_text = []

    for word in text:

        if word.lower() in ["asian", "basis", "solusi", "sufix"]:

            # Save the words without stemming
            stemmed_text.append(word)

        elif word.lower() == "ngutang" or word.lower() == "utang":

            stemmed_text.append("hutang")

        else:

            stemmed_text.append(stemmer.stem(word))

    return stemmed_text

## ✨ 2.1 - Change to lower case

In [76]:
df_sentiment['clean_tweet'] = df_sentiment['tweet'].apply(case_folding_text)

df_sentiment.head()

,sentimen,tweet,clean_tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...,kata @prabowo indonesia tidak dihargai bangsa ...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib...","batuan langka, tasbih jokowi hadiah dari habib..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik....","di era jokowi, ekonomi indonesia semakin baik...."
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p...","bagi sumatera selatan, asian games berdampak p..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...,negara kita ngutang buat bngun infrastruktur y...


In [77]:
display(df_sentiment['tweet'].loc[0])
display(df_sentiment['clean_tweet'].loc[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

'kata @prabowo indonesia tidak dihargai bangsa asing!   berita ini ðÿ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðÿ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21â\xa0â€¦'

In [78]:
display(df_sentiment['tweet'].loc[5])
display(df_sentiment['clean_tweet'].loc[5])

'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundamental. Betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental ekonomi'

'yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. e-sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia, bukan hal fundamental. betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental ekonomi'

In [79]:
display(df_sentiment['tweet'].loc[20])
display(df_sentiment['clean_tweet'].loc[20])

"Assalamu'alaikum.  Saya tahu 1 suara itu sgt berharga dlm upaya memenangkan 02 @prabowo & @sandiuno saya hrs merelakan suara dikarenakan berada di Jkt namun KTP daerah. Semoga suara saya yg tdk disertakan bisa terbalaskan dgn doa. Semoga Allah SWT mengabulkan itu. Aamiin ya Rabb https://twitter.com/sandiuno/status/1117101696696455173Â\xa0â€¦"

"assalamu'alaikum.  saya tahu 1 suara itu sgt berharga dlm upaya memenangkan 02 @prabowo & @sandiuno saya hrs merelakan suara dikarenakan berada di jkt namun ktp daerah. semoga suara saya yg tdk disertakan bisa terbalaskan dgn doa. semoga allah swt mengabulkan itu. aamiin ya rabb https://twitter.com/sandiuno/status/1117101696696455173â\xa0â€¦"

## ✨ 2.2 - Clean up text



🔎 At this stage I deleted several things as follows:
- Mentions (example: @username)
- Hashtags (example: #example123)
- Retweets (example: RT)
- Hyperlinks (example: http://example.com)
- Numbers (example: 12345)
- Twitter pic links (example: pic.twitter.com/example)
- Multiple dots (replaced by one space)
- Newline (replaced by spaces)
- Single quotes (removed)
- Commas (removed)
- Closing brackets (removed)
- Opening brackets (removed)
- Punctuation (all punctuation removed)
- Leading and trailing spaces (removed)

In [80]:
df_sentiment['clean_tweet'] = df_sentiment['clean_tweet'].apply(cleaning_text)

df_sentiment.head()

,sentimen,tweet,clean_tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib...",batuan langka tasbih jokowi hadiah dari habib ...
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik....",di era jokowi ekonomi indonesia semakin baik
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p...",bagi sumatera selatan asian games berdampak pd...
4,negatif,Negara kita ngutang buat bngun infrastruktur y...,negara kita ngutang buat bngun infrastruktur y...


In [81]:
display(df_sentiment['tweet'].loc[0])
display(df_sentiment['clean_tweet'].loc[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

'kata prabowo indonesia tidak dihargai bangsa asing   berita ini  pasti hoax buatan penguasa ya kan rockygerung'

In [82]:
display(df_sentiment['tweet'].loc[5])
display(df_sentiment['clean_tweet'].loc[5])

'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundamental. Betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental ekonomi'

'yg bisikin pak jokowi cm mikirin perputaran duit di golongan tertentu  esports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan hal fundamental  betul yg dikatakan prabowo walau dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi'

## ✨ 2.3 - Replacing slang words

🔎 At this stage I replaced the existing slang words and abbreviations with the slang word dataset that I had loaded in 1.2

In [83]:
df_sentiment['clean_tweet'] = df_sentiment['clean_tweet'].apply(filtering_slang)

df_sentiment.head()

,sentimen,tweet,clean_tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...,kata prabowo indonesia tidak dihargai bangsa a...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib...",batuan langka tasbih jokowi hadiah dari habib ...
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik....",di era jokowi ekonomi indonesia semakin baik
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p...",bagi sumatera selatan asian games berdampak pa...
4,negatif,Negara kita ngutang buat bngun infrastruktur y...,negara kita ngutang buat bangun infrastruktur ...


In [84]:
display(df_sentiment['tweet'].loc[0])
display(df_sentiment['clean_tweet'].loc[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

'kata prabowo indonesia tidak dihargai bangsa asing berita ini pasti isu bohong buatan penguasa iya kan rockygerung'

In [85]:
display(df_sentiment['tweet'].loc[5])
display(df_sentiment['clean_tweet'].loc[5])

'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundamental. Betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental ekonomi'

'yang bisikin pak jokowi cuma memikirkan  perputaran uang di golongan tertentu esports hanya sebagian kecil dari sekian besar penentu pertumbuhan perekonomian indonesia bukan hal fundamental betul yang dikatakan prabowo walau dengan bahas yang sederhana tapi merupakan aspek fundamental ekonomi'

In [86]:
display(df_sentiment['tweet'].loc[20])
display(df_sentiment['clean_tweet'].loc[20])

"Assalamu'alaikum.  Saya tahu 1 suara itu sgt berharga dlm upaya memenangkan 02 @prabowo & @sandiuno saya hrs merelakan suara dikarenakan berada di Jkt namun KTP daerah. Semoga suara saya yg tdk disertakan bisa terbalaskan dgn doa. Semoga Allah SWT mengabulkan itu. Aamiin ya Rabb https://twitter.com/sandiuno/status/1117101696696455173Â\xa0â€¦"

'assalamualaikum saya tahu 1 suara itu sangat berharga dalam upaya memenangkan 02 prabowo sandiuno saya harus merelakan suara dikarenakan berada di jakarta namun kartu tanda penduduk daerah semoga suara saya yang tidak disertakan bisa terbalaskan dengan doa semoga allah swt mengabulkan itu amin  iya rabb'

## ✨ 2.4 - Breaks text into tokens

🔎 At this stage I use tokenization from the NLTK library

In [87]:
df_sentiment['clean_tweet'] = df_sentiment['clean_tweet'].apply(tokenizing_text)

df_sentiment.head()

,sentimen,tweet,clean_tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...,"[kata, prabowo, indonesia, tidak, dihargai, ba..."
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib...","[batuan, langka, tasbih, jokowi, hadiah, dari,..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik....","[di, era, jokowi, ekonomi, indonesia, semakin,..."
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p...","[bagi, sumatera, selatan, asian, games, berdam..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...,"[negara, kita, ngutang, buat, bangun, infrastr..."


In [89]:
display(df_sentiment['tweet'].loc[0])
display(df_sentiment['clean_tweet'].loc[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

['kata',
 'prabowo',
 'indonesia',
 'tidak',
 'dihargai',
 'bangsa',
 'asing',
 'berita',
 'ini',
 'pasti',
 'isu',
 'bohong',
 'buatan',
 'penguasa',
 'iya',
 'kan',
 'rockygerung']

In [90]:
display(df_sentiment['tweet'].loc[5])
display(df_sentiment['clean_tweet'].loc[5])

'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundamental. Betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental ekonomi'

['yang',
 'bisikin',
 'pak',
 'jokowi',
 'cuma',
 'memikirkan',
 'perputaran',
 'uang',
 'di',
 'golongan',
 'tertentu',
 'esports',
 'hanya',
 'sebagian',
 'kecil',
 'dari',
 'sekian',
 'besar',
 'penentu',
 'pertumbuhan',
 'perekonomian',
 'indonesia',
 'bukan',
 'hal',
 'fundamental',
 'betul',
 'yang',
 'dikatakan',
 'prabowo',
 'walau',
 'dengan',
 'bahas',
 'yang',
 'sederhana',
 'tapi',
 'merupakan',
 'aspek',
 'fundamental',
 'ekonomi']

In [91]:
display(df_sentiment['tweet'].loc[20])
display(df_sentiment['clean_tweet'].loc[20])

"Assalamu'alaikum.  Saya tahu 1 suara itu sgt berharga dlm upaya memenangkan 02 @prabowo & @sandiuno saya hrs merelakan suara dikarenakan berada di Jkt namun KTP daerah. Semoga suara saya yg tdk disertakan bisa terbalaskan dgn doa. Semoga Allah SWT mengabulkan itu. Aamiin ya Rabb https://twitter.com/sandiuno/status/1117101696696455173Â\xa0â€¦"

['assalamualaikum',
 'saya',
 'tahu',
 '1',
 'suara',
 'itu',
 'sangat',
 'berharga',
 'dalam',
 'upaya',
 'memenangkan',
 '02',
 'prabowo',
 'sandiuno',
 'saya',
 'harus',
 'merelakan',
 'suara',
 'dikarenakan',
 'berada',
 'di',
 'jakarta',
 'namun',
 'kartu',
 'tanda',
 'penduduk',
 'daerah',
 'semoga',
 'suara',
 'saya',
 'yang',
 'tidak',
 'disertakan',
 'bisa',
 'terbalaskan',
 'dengan',
 'doa',
 'semoga',
 'allah',
 'swt',
 'mengabulkan',
 'itu',
 'amin',
 'iya',
 'rabb']

## ✨ 2.5 - Delete repeated letters

In [ ]:
# df_sentiment['clean_tweet'] = df_sentiment['clean_tweet'].apply(remove_duplicate_letters)

# df_sentiment.head()

## ✨ 2.6 - Remove stop words

In [92]:
df_sentiment['clean_tweet'] = df_sentiment['clean_tweet'].apply(filtering_stopwords)

df_sentiment.head()

,sentimen,tweet,clean_tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...,"[kata, prabowo, indonesia, tidak, dihargai, ba..."
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib...","[batuan, langka, tasbih, jokowi, hadiah, habib..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik....","[era, jokowi, ekonomi, indonesia, semakin, baik]"
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p...","[sumatera, selatan, asian, games, berdampak, e..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...,"[negara, ngutang, buat, bangun, infrastruktur,..."


In [93]:
display(df_sentiment['tweet'].loc[0])
display(df_sentiment['clean_tweet'].loc[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

['kata',
 'prabowo',
 'indonesia',
 'tidak',
 'dihargai',
 'bangsa',
 'asing',
 'berita',
 'isu',
 'bohong',
 'buatan',
 'penguasa',
 'iya',
 'kan',
 'rockygerung']

In [94]:
display(df_sentiment['tweet'].loc[5])
display(df_sentiment['clean_tweet'].loc[5])

'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundamental. Betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental ekonomi'

['bisikin',
 'pak',
 'jokowi',
 'cuma',
 'memikirkan',
 'perputaran',
 'uang',
 'golongan',
 'tertentu',
 'esports',
 'sebagian',
 'kecil',
 'sekian',
 'besar',
 'penentu',
 'pertumbuhan',
 'perekonomian',
 'indonesia',
 'bukan',
 'fundamental',
 'betul',
 'dikatakan',
 'prabowo',
 'bahas',
 'sederhana',
 'merupakan',
 'aspek',
 'fundamental',
 'ekonomi']

In [95]:
display(df_sentiment['tweet'].loc[20])
display(df_sentiment['clean_tweet'].loc[20])

"Assalamu'alaikum.  Saya tahu 1 suara itu sgt berharga dlm upaya memenangkan 02 @prabowo & @sandiuno saya hrs merelakan suara dikarenakan berada di Jkt namun KTP daerah. Semoga suara saya yg tdk disertakan bisa terbalaskan dgn doa. Semoga Allah SWT mengabulkan itu. Aamiin ya Rabb https://twitter.com/sandiuno/status/1117101696696455173Â\xa0â€¦"

['assalamualaikum',
 'tahu',
 '1',
 'suara',
 'sangat',
 'berharga',
 'upaya',
 'memenangkan',
 '02',
 'prabowo',
 'sandiuno',
 'merelakan',
 'suara',
 'dikarenakan',
 'berada',
 'jakarta',
 'kartu',
 'tanda',
 'penduduk',
 'daerah',
 'semoga',
 'suara',
 'tidak',
 'disertakan',
 'terbalaskan',
 'doa',
 'semoga',
 'allah',
 'swt',
 'mengabulkan',
 'amin',
 'iya',
 'rabb']

## ✨ 2.7 - Change the word to its base word

🔎 At this stage I change words into their basic words (stemming) using a sastrawi library

In [96]:
df_sentiment['clean_tweet'] = df_sentiment['clean_tweet'].apply(stemming_text)

df_sentiment.head()

,sentimen,tweet,clean_tweet
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...,"[kata, prabowo, indonesia, tidak, harga, bangs..."
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib...","[batu, langka, tasbih, jokowi, hadiah, habib, ..."
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik....","[era, jokowi, ekonomi, indonesia, makin, baik]"
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p...","[sumatera, selatan, asian, games, dampak, ekon..."
4,negatif,Negara kita ngutang buat bngun infrastruktur y...,"[negara, hutang, buat, bangun, infrastruktur, ..."


# 🎯 Step 3: Recombine the tokenized words into sentences

---



🔎 I recombined the tokens into sentences for exploratory data analysis after preprocessing the dataset

In [97]:
df_sentiment['clean_tweet_sentence']= [' '.join(tokens) for tokens in df_sentiment['clean_tweet']]

df_sentiment.head()

,sentimen,tweet,clean_tweet,clean_tweet_sentence
0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...,"[kata, prabowo, indonesia, tidak, harga, bangs...",kata prabowo indonesia tidak harga bangsa asin...
1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib...","[batu, langka, tasbih, jokowi, hadiah, habib, ...",batu langka tasbih jokowi hadiah habib luthfi ...
2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik....","[era, jokowi, ekonomi, indonesia, makin, baik]",era jokowi ekonomi indonesia makin baik
3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p...","[sumatera, selatan, asian, games, dampak, ekon...",sumatera selatan asian games dampak ekonomi la...
4,negatif,Negara kita ngutang buat bngun infrastruktur y...,"[negara, hutang, buat, bangun, infrastruktur, ...",negara hutang buat bangun infrastruktur pakai ...


# 🎯 Step 4: Save pre-processed data into CSV

---



In [98]:
path =  '/content/drive/MyDrive/INDONESIA AI/Sentiment Analysis/Dataset/df_tweet_after_preprocessing.csv'

df_sentiment.to_csv(path, index=False)